In [1]:
import pandas as pd
import numpy as np

In [2]:
!ls

Untitled.ipynb data


# Загрузка данных

In [3]:
!ls data

gender_submission.csv test.csv              train.csv


In [4]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
gender_submission = pd.read_csv('data/gender_submission.csv')

In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Пример файла с результатом

In [9]:
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# Подготовка данных

In [27]:
y = train.Survived
X_raw = train.copy()
X_raw.drop('Survived', axis=1, inplace=True)
X_raw.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True], dtype=bool)

In [28]:
X_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 76.6+ KB


In [29]:
X_raw.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [47]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

In [52]:
imputer = None

def extract_features(data):
    global imputer, scaler
    X = data.copy()
    X["isMale"] = X.Sex.replace({"male": 1, "female":0})
    X.drop(["Sex", "Cabin", "Ticket", "Name", "PassengerId"], axis=1, inplace=True)
    X = pd.get_dummies(X, columns=['Pclass', 'Embarked'])
        
    columns = X.columns
    
    if imputer is None:
        imputer = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=True)
        imputer.fit(X)
        
    X = pd.DataFrame(imputer.transform(X), columns=columns)
    
    return X

In [53]:
X = extract_features(X_raw)
X.head()

,Age,SibSp,Parch,Fare,isMale,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,22.0,1.0,0.0,7.2500,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,38.0,1.0,0.0,71.2833,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,26.0,0.0,0.0,7.9250,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,35.0,1.0,0.0,53.1000,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,35.0,0.0,0.0,8.0500,1.0,0.0,0.0,1.0,0.0,0.0,1.0


# Обучение

In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [58]:
grid = {'max_depth': np.arange(5, 200, 5)}
gridsearch = GridSearchCV(DecisionTreeClassifier(), grid, scoring='accuracy', cv=5)

In [59]:
%%time
gridsearch.fit(X, y)

CPU times: user 689 ms, sys: 4.69 ms, total: 694 ms
Wall time: 693 ms


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': array([  5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,
        70,  75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125, 130,
       135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [60]:
gridsearch.best_params_

{'max_depth': 5}

In [64]:
clf = DecisionTreeClassifier(max_depth=gridsearch.best_params_['max_depth'])
clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

# Граф классификатора
http://www.webgraphviz.com

In [78]:
from sklearn.tree import export_graphviz

print(export_graphviz(clf, out_file=None, filled=True, feature_names=list(X.columns)))

digraph Tree {
node [shape=box, style="filled", color="black"] ;
0 [label="isMale <= 0.5\ngini = 0.473\nsamples = 891\nvalue = [549, 342]", fillcolor="#e5813960"] ;
1 [label="Pclass_3 <= 0.5\ngini = 0.383\nsamples = 314\nvalue = [81, 233]", fillcolor="#399de5a6"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="Age <= 2.5\ngini = 0.1\nsamples = 170\nvalue = [9, 161]", fillcolor="#399de5f1"] ;
1 -> 2 ;
3 [label="Parch <= 1.5\ngini = 0.5\nsamples = 2\nvalue = [1, 1]", fillcolor="#e5813900"] ;
2 -> 3 ;
4 [label="gini = 0.0\nsamples = 1\nvalue = [0, 1]", fillcolor="#399de5ff"] ;
3 -> 4 ;
5 [label="gini = 0.0\nsamples = 1\nvalue = [1, 0]", fillcolor="#e58139ff"] ;
3 -> 5 ;
6 [label="Fare <= 28.856\ngini = 0.091\nsamples = 168\nvalue = [8, 160]", fillcolor="#399de5f2"] ;
2 -> 6 ;
7 [label="Fare <= 28.231\ngini = 0.182\nsamples = 69\nvalue = [7, 62]", fillcolor="#399de5e2"] ;
6 -> 7 ;
8 [label="gini = 0.161\nsamples = 68\nvalue = [6, 62]", fillcolor="#399de5e6"] ;
7 -

![tree](./tree.svg)

# Предсказания

In [62]:
X_test = extract_features(test)
X_test.head()

,Age,SibSp,Parch,Fare,isMale,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,34.5,0.0,0.0,7.8292,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,47.0,1.0,0.0,7.0000,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,62.0,0.0,0.0,9.6875,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,27.0,0.0,0.0,8.6625,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4,22.0,1.0,1.0,12.2875,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [66]:
predictions = clf.predict(X_test)

In [73]:
results = pd.DataFrame(list(zip(test.PassengerId, predictions)), columns=['PassengerId', 'Survived'])
results.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [74]:
results.to_csv('submission.csv', index=False)

![Alt text](./kaggle.png)